## Rules

1. All mathematical expressions should be written in **LaTeX** for better clarity and formatting.
2. Ensure that the entire notebook can execute seamlessly from start to finish without encountering errors.
3. Focus on optimizing the runtime of the code wherever possible to enhance performance.

## Notation

- $c$: The optimal constant model.  
- $y_i$: The target values in the dataset.  
- $w_i$: The weights associated with the loss function.
- $q$: Quantile value in range $[0, 1]$.

# Важно! О формате сдачи

* **При решении ноутбука используйте данный шаблон. Не нужно удалять текстовые ячейки c разметкой частей ноутбука и формулировками заданий. Добавлять свои ячейки, при необходимости, конечно можно**
* **Везде, где в формулровке задания есть какой-либо вопрос (или просьба вывода), необходимо прописать ответ в ячейку (код или markdown).**
* **Наличие кода решения (или аналитического решения - в зависимости от задачи) обязательно. Письменные ответы на вопросы без сопутствующего кода/аналитического решения оцениваются в 0 баллов.**

## Problem 1

### Description

Consider the loss function:

\begin{align}
L = \sum w_i \cdot \left( \log(y_i) - \log(c) \right)^2
\end{align}

where:

- $\sum w_i = 1$

#### Tasks

1. **Analytically find the best constant $c$** for the given loss function.
2. **Determine the name of the aggregation of $y_i$'s** at the end if $w_1 = w_2 = \dots = w_n$.

### Solution


$$
L = \sum w_i \cdot \left( \log(y_i) - \log(c) \right)^2
$$

---

   $$
   L = \sum w_i \cdot \left( \log(y_i) - x \right)^2
   $$


   $$
   \frac{dL}{dx} = -2 \sum w_i \cdot \left( \log(y_i) - x \right)
   $$


   $$
   -2 \sum w_i \cdot \left( \log(y_i) - x \right) = 0 \implies x = \sum w_i \log(y_i)
   $$


   $$
   \log(c) = \sum w_i \log(y_i) \implies c = \exp\left(\sum w_i \log(y_i)\right)
   $$

---


Если \( w_1 = w_2 = \dots = w_n = \frac{1}{n} \), то:
$$
c = \exp\left(\frac{1}{n} \sum \log(y_i)\right) = \prod y_i^{1/n}
$$

---

**Final Answers:**  
1. Оптимальная константа:  
   $$\boxed{c = \exp\left(\sum w_i \log(y_i)\right)}$$  
2. При равных весах агрегация называется средним геометрическим

In [5]:
import numpy as np

# 1. Аналитическое решение
def find_oconstant(y, w=None):
    y = np.array(y)
    if w is None:
        w = np.ones_like(y) / len(y)
    else:
        w = np.array(w) / np.sum(w)
    
    c = np.exp(np.sum(w * np.log(y)))
    return c

# 2. Проверка для случая равных весов
y = np.array([1, 2, 3, 4, 5])
c_equal = find_oconstant(y)  # равные веса
geom_mean = np.exp(np.mean(np.log(y)))  # геометрическое среднее

print("Ответ:")
print("1. Оптимальная константа c = exp(∑(w_i * log(y_i)))",c_equal)
print("2. При равных весах (w_i = 1/n) получаем геометрическое среднее значений y_i", geom_mean)

Ответ:
1. Оптимальная константа c = exp(∑(w_i * log(y_i))) 2.0
2. При равных весах (w_i = 1/n) получаем геометрическое среднее значений y_i 2.0


## Problem 2

### Description

Consider the **quantile loss function** $L$, and prove that the optimal constant $c$ corresponds to the quantile $q$ of the data $y_1, \dots, y_n$.

The quantile loss function is defined as:

\begin{align}
L =
\begin{cases}
q \cdot (y_i - c), & \text{if } y_i \geq c \\
(1-q) \cdot (c - y_i), & \text{if } y_i < c
\end{cases}
\end{align}

**Hint**:

Proof for optimal MAE constant on [this page](https://ds100.org/course-notes/constant_model_loss_transformations/loss_transformations.html).



### Solution

### Solution

---

$$
L(c, y_i) = 
\begin{cases} 
q \cdot (y_i - c), & \text{if } y_i \geq c \\
(1 - q) \cdot (c - y_i), & \text{if } y_i < c 
\end{cases}
$$
$$
L = \sum_{i=1}^n L(c, y_i)
$$

---

  $$
  \frac{\partial L}{\partial c} = (1 - q) \cdot (\text{number of } y_i < c) - q \cdot (\text{number of } y_i \geq c)
  $$

  $$
  \frac{\partial L}{\partial c} = (1 - q)k - q(n - k)
  $$

---

$$
(1 - q)k - q(n - k) = 0 \implies k = qn
$$


---

#### **Final Answer**  
The optimal constant \( c \) that minimizes the quantile loss function is the:  
$$
\boxed{q\text{-th quantile}} \text{ of } y_1, \dots, y_n.
$$

---


## Problem 3

In [3]:
import numpy as np

def bruteforce_constant(y: np.ndarray, loss_func: callable, tol: float) -> float:
    cv = np.arange(y.min(), y.max() + tol, tol)
    bc = cv[0]
    ml = float('inf')
    for c in cv:
        cr = loss_func(y, c)
        if cr < ml:
            ml = cr
            bc = c
    return bc

def loss(y, c):
    return np.sum(np.log(np.cosh(y - c)))

y = np.array([1, 2, 3, 4, 55, 99, 100])
oc = bruteforce_constant(y, loss, tol=0.001)
ml = loss(y, oc)
print(f"The optimal constant c is: {oc}")
print(f"The minimum loss is: {ml}")

The optimal constant c is: 4.1969999999996475
The minimum loss is: 243.96167948452023


## Problem 4

In [5]:
from scipy.optimize import minimize
import numpy as np

def minimize_constant(y: np.ndarray, loss_func: callable, tol: float) -> float:
    ic = np.median(y)
    

    def objective(c_array):
        return loss_func(y, c_array[0])
    
    res = minimize(objective, x0=[ic], method='BFGS', tol=tol)
    

    return res.x[0]

def loss(y, c):
    return np.sum(np.log(np.cosh(y - c)))

y = np.array([1, 2, 3, 4, 55, 99, 100])
oc = minimize_constant(y, loss, tol=0.001)
ml = loss(y, oc)
print(f"The optimal constant c is: {oc}")
print(f"The minimum loss is: {ml}")

The optimal constant c is: 4.197202525934449
The minimum loss is: 243.9616794082023


# Problem 5

## Description
In a multiclass classification problem, we often compare a model’s performance with a simple baseline. Two baseline approaches are:

1. **Always predict the most frequent class** (also known as the “constant baseline”).  
2. **Randomly sample a class** for every object with probabilities proportional to the class frequencies in the training set.

The goal of this task is to determine which of these two approaches yields a higher **Accuracy** on a given dataset.

You are provided with:  
1. A set of objects \\( X \\) and their true class labels \\( y \\), where \\( y \in \{C_1, C_2, \dots, C_k\} \\).  
2. The frequencies (or proportions) of each class in the training data.

You need to:  
1. **Explain** how to implement these two baseline approaches (always predicting the most frequent class vs. sampling a class according to its frequency).  
2. **Calculate** the expected Accuracy for each approach.  
3. **Compare** the ress to determine which method is more likely to produce a higher Accuracy.

## Solution


\[
p(C_m) \quad \text{vs} \quad \sum_{j=1}^{k} p(C_j)^2
\]

- Since \( p(C_m) \) is the maximum class probability, we analyze whether:
  \[
  p(C_m) \geq \sum_{j=1}^{k} p(C_j)^2
  \]
- Using Jensen's inequality for convex functions (\( f(x) = x^2 \) is convex for \( x \geq 0 \)), we apply the inequality:
  \[
  \sum_{j=1}^{k} p(C_j)^2 \leq p(C_m)
  \]


---

### Final Answer:
In most cases, always predicting the most frequent class is more likely to yield a higher Accuracy, especially when the class distribution is imbalanced.